In [2]:
import cntk
import numpy
import cv2
from matplotlib import pyplot as plt
from PIL import Image

def create_reader(path, is_training, input_dim, num_label_classes):
    labelStream = cntk.io.StreamDef(field='labels', shape=num_label_classes, is_sparse=False)
    featureStream = cntk.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(labels=labelStream, features=featureStream))
    return cntk.io.MinibatchSource(deserailizer, randomize=is_training, max_sweeps=cntk.io.INFINITELY_REPEAT if is_training else 1)

def load_model(path):
    model = cntk.ops.functions.load_model(path)
    return model

def predict(model, data):
    return model.eval(data[0])

def do_predict(model_path, data_path):
    model = load_model(model_path)
    reader = create_reader(data_path, False, 3, 3)
    input_map = {
        #label: reader.streams.labels,
        input: reader.streams.features
    }
    data = reader.next_minibatch(1, input_map=input_map)
    
    feature_data = data[input].asarray()[0]
    print(feature_data)
    #save_image(feature_data[0], "feature_image.png")
    
    #label_data = data[label].asarray()[0]
    #print(label_data)
    
    return predict(model, feature_data)

def save_image(arr, name):
    predict_R = arr[0]
    predict_G = arr[1]
    predict_B = arr[2]
    im = Image.new("RGB", (128, 128))
    pix = im.load()
    for x in range(128):
        for y in range(128):
            pix[x, y] = (predict_R, predict_G, predict_B)
            
    im.save(name, "PNG")
    print("save " + name + " success")

model_path = "./saved_model/set-8.model"
data_path = "../Data/evaluate-data-3.txt"
predict_value = do_predict(model_path, data_path)
predict_value = predict_value.round()
print(predict_value)
#save_image(predict_value[0], "predicted_image.png")




[[ 135.   71.   23.]]
[[ 112.   73.   34.]]
